### random vs info_gain vs chi2

In [1]:
import numpy as np
import os
import pandas as pd
    
geno = np.loadtxt('/datasets/cs284-sp21-A00-public/ps2/precomputed/ps2_pca.genotypes.tab') # (28622 SNPs, 2504 people)
pop_6 = np.load('pop_6.npy')

In [2]:
train = geno[:,:2000]
test = geno[:,2000:]
train_Y = np.array(pop_6[:2000])
test_Y = np.array(pop_6[2000:])

In [3]:
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.ensemble import  AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB

import time


def simple_SVM(X,Y,test_X,test_Y,method='svm_linear'):
    time0=time.time()
    if method=='svm_linear' or method==0:
        clf = svm.SVC(kernel='linear')
    elif method=='KNeighbors' or method==1:
        clf = KNeighborsClassifier()
    elif method=='LogisticRegression' or method==2:
        clf = LogisticRegression(penalty='l2')
    elif method=='RandomForest' or method==3:
        clf = RandomForestClassifier(n_estimators=8)
    elif method=='DecisionTree' or method==4:
        clf = tree.DecisionTreeClassifier()
    elif method=='AdaBoost' or method==5:
        clf = AdaBoostClassifier()
    elif method=='Bayes' or method==6:
        clf = MultinomialNB(alpha=0.01)
    else:
        print('Not implied!')
        return None
    
    clf.fit(X,Y)
    
    print('Train time:',int(time.time()-time0))
    
    print('train acc:',clf.score(X, Y))
    print('test acc:',clf.score(test_X, test_Y))

In [4]:
## random
import random
for num in [10,50]:
    print('snps:',num)
    rl = random.sample(range(28622), num)
    simple_SVM(train[rl,:].T,train_Y,test[rl,:].T,test_Y)
    simple_SVM(train[rl,:].T,train_Y,test[rl,:].T,test_Y,1)
    simple_SVM(train[rl,:].T,train_Y,test[rl,:].T,test_Y,2)
    simple_SVM(train[rl,:].T,train_Y,test[rl,:].T,test_Y,3)
    simple_SVM(train[rl,:].T,train_Y,test[rl,:].T,test_Y,4)
    simple_SVM(train[rl,:].T,train_Y,test[rl,:].T,test_Y,5)
    simple_SVM(train[rl,:].T,train_Y,test[rl,:].T,test_Y,6)
    print('')

snps: 10
Train time: 0
train acc: 0.5565
test acc: 0.4801587301587302
Train time: 0
train acc: 0.6035
test acc: 0.37103174603174605
Train time: 0
train acc: 0.5605
test acc: 0.4662698412698413
Train time: 0


/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


train acc: 0.753
test acc: 0.3968253968253968
Train time: 0
train acc: 0.7645
test acc: 0.3888888888888889
Train time: 0
train acc: 0.3395
test acc: 0.3611111111111111
Train time: 0
train acc: 0.5185
test acc: 0.4226190476190476

snps: 50
Train time: 0
train acc: 0.8565
test acc: 0.6746031746031746
Train time: 0
train acc: 0.7915
test acc: 0.5654761904761905


/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Train time: 7
train acc: 0.8385
test acc: 0.6309523809523809
Train time: 0
train acc: 0.99
test acc: 0.5952380952380952
Train time: 0
train acc: 1.0
test acc: 0.4742063492063492
Train time: 0
train acc: 0.353
test acc: 0.23809523809523808
Train time: 0
train acc: 0.8035
test acc: 0.7003968253968254



In [5]:
## info_gain
from collections import defaultdict
from math import log

def calcShannonEnt(Y):
    numEntries = len(Y)
    labelCounts = defaultdict(int)
    for label in Y:
        labelCounts[label] += 1
    shannonEnt = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key])/numEntries
        shannonEnt -= prob * log(prob, 2)
    return shannonEnt

def calcInfoGain(dataset,Y):
    baseEntropy = calcShannonEnt(Y)
    InfoGain_list = []
    for i in range(dataset.shape[0]):
        subset_list = [[],[],[]]
        for j in range(dataset.shape[1]):
            subset_list[int(dataset[i,j])].append(j)
        newEntropy = 0.0
        splitInfo = 0.0
        for value in [0,1,2]:
            if len(subset_list[value])==0:
                continue
            prob = len(subset_list[value])/float(dataset.shape[1])
            newEntropy += prob * calcShannonEnt(Y[subset_list[value]])
            splitInfo += -prob * log(prob, 2)
        infoGain = baseEntropy - newEntropy
        if (splitInfo == 0):
            continue
        infoGainRatio = infoGain / splitInfo
        InfoGain_list.append(infoGainRatio)
    return InfoGain_list

InfoGain_list = calcInfoGain(train,train_Y)
rank_infoGain = np.array(InfoGain_list).argsort()

for num in [10,50]:
    
    print('snps:',num)
    simple_SVM(train[rank_infoGain[-num:],:].T,train_Y,test[rank_infoGain[-num:],:].T,test_Y,0)
    simple_SVM(train[rank_infoGain[-num:],:].T,train_Y,test[rank_infoGain[-num:],:].T,test_Y,1)
    simple_SVM(train[rank_infoGain[-num:],:].T,train_Y,test[rank_infoGain[-num:],:].T,test_Y,2)
    simple_SVM(train[rank_infoGain[-num:],:].T,train_Y,test[rank_infoGain[-num:],:].T,test_Y,3)
    simple_SVM(train[rank_infoGain[-num:],:].T,train_Y,test[rank_infoGain[-num:],:].T,test_Y,4)
    simple_SVM(train[rank_infoGain[-num:],:].T,train_Y,test[rank_infoGain[-num:],:].T,test_Y,5)
    simple_SVM(train[rank_infoGain[-num:],:].T,train_Y,test[rank_infoGain[-num:],:].T,test_Y,6)
    print('')

snps: 10
Train time: 0
train acc: 0.5655
test acc: 0.5138888888888888
Train time: 0
train acc: 0.552
test acc: 0.375
Train time: 0
train acc: 0.5775
test acc: 0.4861111111111111
Train time: 0
train acc: 0.6915
test acc: 0.48214285714285715
Train time: 0
train acc: 0.702
test acc: 0.46825396825396826


/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Train time: 0
train acc: 0.318
test acc: 0.36507936507936506
Train time: 0
train acc: 0.5245
test acc: 0.4662698412698413

snps: 50
Train time: 0
train acc: 0.8325
test acc: 0.7222222222222222
Train time: 0
train acc: 0.7815
test acc: 0.6091269841269841


/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Train time: 7
train acc: 0.8185
test acc: 0.7242063492063492
Train time: 0
train acc: 0.9825
test acc: 0.6349206349206349
Train time: 0
train acc: 1.0
test acc: 0.5
Train time: 0
train acc: 0.358
test acc: 0.16666666666666666
Train time: 0
train acc: 0.788
test acc: 0.7281746031746031



In [6]:
## chi2
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

for num in [10,50,100,500,1000]:
    print('snps:',num)
    chi2_selection = SelectKBest(chi2, k=num).fit(train.T, np.array(train_Y))
    X_select_index = chi2_selection.get_support(indices=True)
    
    simple_SVM(train[X_select_index,:].T,train_Y,test[X_select_index,:].T,test_Y,0)
    simple_SVM(train[X_select_index,:].T,train_Y,test[X_select_index,:].T,test_Y,1)
    simple_SVM(train[X_select_index,:].T,train_Y,test[X_select_index,:].T,test_Y,2)
    simple_SVM(train[X_select_index,:].T,train_Y,test[X_select_index,:].T,test_Y,3)
    simple_SVM(train[X_select_index,:].T,train_Y,test[X_select_index,:].T,test_Y,4)
    simple_SVM(train[X_select_index,:].T,train_Y,test[X_select_index,:].T,test_Y,5)
    simple_SVM(train[X_select_index,:].T,train_Y,test[X_select_index,:].T,test_Y,6)
    print('')

snps: 10
Train time: 0
train acc: 0.661
test acc: 0.5972222222222222
Train time: 0
train acc: 0.5915
test acc: 0.5297619047619048
Train time: 0
train acc: 0.6685
test acc: 0.5833333333333334
Train time: 0
train acc: 0.7205


/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


test acc: 0.5615079365079365
Train time: 0
train acc: 0.723
test acc: 0.5555555555555556
Train time: 0
train acc: 0.525
test acc: 0.44841269841269843
Train time: 0
train acc: 0.543
test acc: 0.48214285714285715

snps: 50
Train time: 0
train acc: 0.8625
test acc: 0.7400793650793651
Train time: 0
train acc: 0.8245
test acc: 0.6924603174603174


/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Train time: 7
train acc: 0.8605
test acc: 0.7400793650793651
Train time: 0
train acc: 0.964
test acc: 0.6805555555555556
Train time: 0
train acc: 0.976
test acc: 0.6369047619047619
Train time: 0
train acc: 0.6305
test acc: 0.45436507936507936
Train time: 0
train acc: 0.778
test acc: 0.7043650793650794

snps: 100
Train time: 0
train acc: 0.903
test acc: 0.7321428571428571
Train time: 0
train acc: 0.8255
test acc: 0.6984126984126984


...

Train time: 9
train acc: 0.892
test acc: 0.7242063492063492
Train time: 0
train acc: 0.9865
test acc: 0.7202380952380952
Train time: 0
train acc: 0.9985
test acc: 0.6388888888888888
Train time: 0
train acc: 0.5505
test acc: 0.44047619047619047
Train time: 0
train acc: 0.7995
test acc: 0.7301587301587301

snps: 500
Train time: 0
train acc: 1.0
test acc: 0.7837301587301587
Train time: 0
train acc: 0.891
test acc: 0.8194444444444444


/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Train time: 15
train acc: 1.0
test acc: 0.7936507936507936
Train time: 0
train acc: 0.994
test acc: 0.751984126984127
Train time: 0
train acc: 1.0
test acc: 0.7123015873015873
Train time: 0
train acc: 0.609
test acc: 0.5257936507936508
Train time: 0
train acc: 0.9035
test acc: 0.8511904761904762

snps: 1000
Train time: 0
train acc: 1.0
test acc: 0.8492063492063492
Train time: 0
train acc: 0.8945
test acc: 0.8293650793650794


/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Train time: 21
train acc: 1.0
test acc: 0.8392857142857143
Train time: 0
train acc: 0.9965
test acc: 0.8154761904761905
Train time: 0
train acc: 1.0
test acc: 0.7301587301587301
Train time: 1
train acc: 0.579
test acc: 0.501984126984127
Train time: 0
train acc: 0.9345
test acc: 0.8928571428571429



Format:  
snp number  
(left: random; middle: info_gain; right: chi2)  
0 svm_linear  
1 KNeighbors  
2 LogisticRegression  
3 RandomForest  
4 DecisionTree  
5 AdaBoost  
6 Bayes  


10  
0.44 0.51 0.59  
0.32 0.37 0.52  
0.44 0.48 0.58  
0.37 0.46 0.58  
0.34 0.46 0.54  
0.18 0.36 0.44  
0.41 0.46 0.48  

50  
0.64 0.72 0.74  
0.57 0.60 0.69  
0.66 0.72 0.74  
0.51 0.61 0.69  
0.42 0.50 0.64  
0.51 0.16 0.45  
0.65 0.72 0.70  

100  
0.71 0.75 0.73  
0.69 0.70 0.69  
0.72 0.76 0.72  
0.62 0.63 0.72  
0.50 0.52 0.65  
0.22 0.29 0.44  
0.78 0.79 0.73

500  
0.79 0.79 0.78  
0.84 0.84 0.81  
0.79 0.81 0.79  
0.73 0.72 0.77  
0.62 0.64 0.72  
0.33 0.39 0.82  
0.87 0.89 0.85  

1000  
0.82 0.82 0.84  
0.84 0.87 0.82  
0.83 0.85 0.83  
0.75 0.75 0.78  
0.66 0.67 0.75  
0.53 0.54 0.50  
0.87 0.90 0.89